In [1]:
%reload_ext jarvis

Log file storage activated
ERROR, WARNING and INFO messages display activated
Diagram file storage activated
Open Modelica simulation activated
plantUml.jar is not up-to-date, see latest release https://github.com/plantuml/plantuml/releases/latest
Hello casteran
I am Jarvis. How may I assist you?


In [2]:
%retrieve_pkg_version


ipython==8.27.0
lxml==5.3.0
notebook==7.2.2
plantuml==0.3.0
jarvis4se==1.4.0
pandas==1.4.1
requests==2.32.3
nltk==3.8.1
PyZMQ==26.2.0
OMPython==3.5.1 
python==3.10.1


from https://www.depslink.fr/wp-content/uploads/2024/09/DEPS_a_model-_and_property-based_language_for_syst.pdf

The second problem is described in [1, 56] and it deals with __Unmanned Aerial Vehicle (UAV)__ also called drone. The degree of autonomy of these systems evolves progressively from the tracking of a trajectory defined by an operator to a complete autonomy.

In the case of trajectory tracking, a key functionality of the UAV is its ability __to detect obstacles__ and __plan a new trajectory__ that will allow it to avoid the UAV while continuing its path to the desired destination.

This function composed of __obstacle detection__, __decision making__ and __calculation of a new trajectory__ must be ensured by the drone alone, without intervention from the ground station, in order to gain in efficiency.

It must also be optimized to adapt to the drone’s energy resources.

The __obstacle detection and avoidance functionality__ is provided by a __platform (HW) using a heterogeneous multi-core architecture (SHMC processor)__, which allows the UAV to have a high computing power while reducing its energy consumption (compared to a single-core or homogeneous multi-core processor).

The SHMC processor is composed of cores with different computing power but using the same instruction set.

A core can thus be allocated to any software task, without the need for recompilation. The cores can also be shut down when they are not allocated to
any task, without the need to be kept in a sleep mode.

On this type of processor we generally find a set of more powerful cores (Big Core) and a set of less powerful cores (Little Core).

The platform used for the case study is an Exynos 5422 processor, composed of __four Cortex A15 cores called "BigCore"__ and __four Cortex A7 cores called "LittleCore"__.

A task is characterized by:
* Name
* CoreNumber :
    * 1: BigCore1
    * 2: BigCore2
    * 3: BigCore3
    * 4: BigCore4
    * 5: LittleCore1
    * 6: LittleCore2
    * 7: LittleCore3
    * 8: LittleCore4
* A memory size required for its execution in KB, including the heap size (MemorySize).
* WCET (Worst Case Execution Time) for Big Core (WCETBC)
* WCET (Worst Case Execution Time) for Little Core (WCETLC).
* A period (Period).
* Rate for Big Core : WCETBC / Period (bcRate)
* Rate for Little Core : WCETLC / Period (lcRate)

Req 1: A task Ti is deployed on one and only one core

Req 2: Some tasks can only be performed by certain cores according to:
* T1 : 1 ou 5
* T2 : 2 ou 6
* T6 : 3 ou 7

Req 3 : Some tasks must be performed by the same core or by two different cores

This type of constraint is noted OnSameCore or OnDifferentCore. These constraints group together tasks sharing a large amount of data (images
stored after acquisition) and separate other tasks for parallelism or reliability (in case of failure of a core). 

Thus, Tasks T1 and T3 as well as Task T2 and T4 must necessarily be deployed on the same core.

Similarly, Task T0 and T1 must necessarily be deployed on different cores

Req 4: The amount of RAM memory available in each core to execute the tasks assigned to it must not be exceeded. 

Thus, the sum of the memory sizes required to perform the tasks assigned to a core must be less than or equal to the memory size of the considered core.
Req 5:The utilization rate of each core, which must not exceed 100%.

Thus the sum of the quotients of WCET by the period for each task assigned to a core must be less than or equal to 1.

from: https://depozit.isae.fr/theses/2017/2017_Leserf_Patrick.pdf

A BigCore is characterized by:
* frequency: 2 GHz
* max memory : 2 000 KB
* max power consumption: 3 Watts

A LittleCore is characterized by:
* frequency: 1.4 GHz
* max memory : 512 KB
* max power consumption: 1 Watt

Les temps WCET ont été mesurés sur la carte ODROID en instrumentant le code embarqué avec un timer HW, sur une série de 1000 images différentes, et
avec une consigne d’acquisition de 20 images par seconde.

In [3]:
%%jarvis
with sw_uav
"Hardware Platform" extends "Functional element"
"Core" extends "Functional element"

"MemorySize (KB)" is an attribute
"WCETBC (ms)" is an attribute
"WCETLC (ms)" is an attribute
"Period (ms)" is an attribute
"CoreNumber" is an attribute
"MemorySizeMax (KB)" is an attribute
"PowerConsumptionMax (W)" is an attribute

Creating sw_uav.xml !
Hardware Platform is a type extending Functional element
Core is a type extending Functional element
MemorySize (KB) is an attribute
WCETBC (ms) is an attribute
WCETLC (ms) is an attribute
Period (ms) is an attribute
CoreNumber is an attribute
MemorySizeMax (KB) is an attribute
PowerConsumptionMax (W) is an attribute
sw_uav.xml updated


In [4]:
%%jarvis
with sw_uav
"UAV - Unnamed Aerial Vehicle" is a functional element.

# T0 : "F1 - To capture frame", coreId, 300, 10, 30, 80, bcRate, lcRate
"F1 - To capture frame" is a function.
The "MemorySize (KB)" of F1 is 300.
The "WCETBC (ms)" of F1 is 10.
The "WCETLC (ms)" of F1 is 30.
The "Period (ms)" of F1 is 80.

# T1 : "F2 - To identify picture", coreId, 100, 20, 45, 90, bcRate, lcRate
"F2 - To identify picture" is a function.
The "MemorySize (KB)" of F2 is 100.
The "WCETBC (ms)" of F2 is 20.
The "WCETLC (ms)" of F2 is 45.
The "Period (ms)" of F2 is 90.

# T2 : "F3 - To appply Filter 1", coreId, 200, 30, 55, 100, bcRate, lcRate
"F3 - To appply Filter 1" is a function.
The "MemorySize (KB)" of F3 is 200.
The "WCETBC (ms)" of F3 is 30.
The "WCETLC (ms)" of F3 is 55.
The "Period (ms)" of F3 is 100.

# T3 : "F4 - To appply Filter 2", coreId, 100, 30, 60, 100, bcRate, lcRate
"F4 - To appply Filter 2" is a function.
The "MemorySize (KB)" of F4 is 100.
The "WCETBC (ms)" of F4 is 30.
The "WCETLC (ms)" of F4 is 60.
The "Period (ms)" of F4 is 100.

# T4 : "F5 - To identify objects", coreId, 200, 20, 55, 80, bcRate, lcRate
"F5 - To identify objects" is a function.
The "MemorySize (KB)" of F5 is 200.
The "WCETBC (ms)" of F5 is 20.
The "WCETLC (ms)" of F5 is 55.
The "Period (ms)" of F5 is 80.

# T5 : "F6 - To determine region occupancy", coreId, 200, 15, 45, 70, bcRate, lcRate
"F6 - To determine region occupancy" is a function.
The "MemorySize (KB)" of F6 is 200.
The "WCETBC (ms)" of F6 is 15.
The "WCETLC (ms)" of F6 is 45.
The "Period (ms)" of F6 is 70.

# T6 : "F7 - To predict trajectory", coreId, 200, 30, 45, 90, bcRate, lcRate
"F7 - To predict trajectory" is a function.
The "MemorySize (KB)" of F7 is 200.
The "WCETBC (ms)" of F7 is 30.
The "WCETLC (ms)" of F7 is 45.
The "Period (ms) of F7 is 90.

# T7 : "F8 - To detect obstacle", coreId, 200, 35, 50, 100, bcRate, lcRate
"F8 - To detect obstacle" is a function.
The "MemorySize (KB)" of F8 is 200.
The "WCETBC (ms)" of F8 is 35.
The "WCETLC (ms)" of F8 is 50.
The "Period (ms)" of F8 is 100.

# T8 : "F9 - To avoid obstacle", coreId, 300, 20, 40, 90, bcRate, lcRate
"F9 - To avoid obstacle" is a function
The "MemorySize (KB)" of F9 is 300.
The "WCETBC (ms)" of F9 is 20.
The "WCETLC (ms)" of F9 is 40.
The "Period (ms)" of F9 is 90.

# T9 : "F10 - To calculate new trajectory", coreId, 400, 40, 60, 100, bcRate, lcRate
"F10 - To calculate new trajectory" is a function.
The "MemorySize (KB)" of F10 is 400.
The "WCETBC (ms)" of F10 is 40.
The "WCETLC (ms)" of F10 is 60.
The "Period (ms)" of F10 is 100.

sw_uav.xml parsed
UAV - Unnamed Aerial Vehicle is a Functional element
F1 - To capture frame is a Function
F2 - To identify picture is a Function
F3 - To appply Filter 1 is a Function
F4 - To appply Filter 2 is a Function
F5 - To identify objects is a Function
F6 - To determine region occupancy is a Function
F7 - To predict trajectory is a Function
F8 - To detect obstacle is a Function
F9 - To avoid obstacle is a Function
F10 - To calculate new trajectory is a Function
Attribute MemorySize (KB) for F1 - To capture frame with value 300
Attribute WCETBC (ms) for F1 - To capture frame with value 10
Attribute WCETLC (ms) for F1 - To capture frame with value 30
Attribute Period (ms) for F1 - To capture frame with value 80
Attribute MemorySize (KB) for F2 - To identify picture with value 100
Attribute WCETBC (ms) for F2 - To identify picture with value 20
Attribute WCETLC (ms) for F2 - To identify picture with value 45
Attribute Period (ms) for F2 - To identify picture with value 90
Attribut

In [5]:
%%jarvis
with sw_uav
UAV allocates F1, F2, F3, F4, F5, F6, F7, F8, F9, F10

sw_uav.xml parsed
Function F1 - To capture frame is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle
Function F2 - To identify picture is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle
Function F3 - To appply Filter 1 is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle
Function F4 - To appply Filter 2 is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle
Function F5 - To identify objects is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle
Function F6 - To determine region occupancy is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle
Function F7 - To predict trajectory is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle
Function F8 - To detect obstacle is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle
Function F9 - To avoid obstacle is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle
Function F10 - To calculate new trajectory is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle

In [6]:
%%jarvis
with sw_uav
"SHMC - Heterogeneous multi-core architecture platform" is an Hardware Platform
UAV is composed of SHMC
SHMC allocates F1, F2, F3, F4, F5, F6, F7, F8, F9, F10

sw_uav.xml parsed
SHMC - Heterogeneous multi-core architecture platform is a Hardware Platform
UAV - Unnamed Aerial Vehicle is composed of SHMC - Heterogeneous multi-core architecture platform
Function F1 - To capture frame is allocated to FunctionalElement SHMC - Heterogeneous multi-core architecture platform
Function F2 - To identify picture is allocated to FunctionalElement SHMC - Heterogeneous multi-core architecture platform
Function F3 - To appply Filter 1 is allocated to FunctionalElement SHMC - Heterogeneous multi-core architecture platform
Function F4 - To appply Filter 2 is allocated to FunctionalElement SHMC - Heterogeneous multi-core architecture platform
Function F5 - To identify objects is allocated to FunctionalElement SHMC - Heterogeneous multi-core architecture platform
Function F6 - To determine region occupancy is allocated to FunctionalElement SHMC - Heterogeneous multi-core architecture platform
Function F7 - To predict trajectory is allocated to FunctionalElement 

In [7]:
%%jarvis
with sw_uav
"BC1 - BigCore1" is a Core.
The "CoreNumber" of BC1 is 1.
The "MemorySizeMax (KB)" of BC1 is 2000.
The "PowerConsumptionMax (W)" of BC1 is 3.

"BC2 - BigCore2" is a Core.
The "CoreNumber" of BC2 is 2.
The "MemorySizeMax (KB)" of BC2 is 2000.
The "PowerConsumptionMax (W)" of BC2 is 3.

"BC3 - BigCore3" is a Core.
The "CoreNumber" of BC3 is 3.
The "MemorySizeMax (KB)" of BC3 is 2000.
The "PowerConsumptionMax (W)" of BC3 is 3.

"BC4 - BigCore4" is a Core.
The "CoreNumber" of BC4 is 4.
The "MemorySizeMax (KB)" of BC4 is 2000.
The "PowerConsumptionMax (W)" of BC4 is 3.

"LC1 - LittleCore1" is a Core.
The "CoreNumber" of LC1 is 5.
The "MemorySizeMax (KB)" of LC1 is 512.
The "PowerConsumptionMax (W)" of LC1 is 1.

"LC2 - LittleCore2" is a Core.
The "CoreNumber" of LC2 is 6.
The "MemorySizeMax (KB)" of LC2 is 512.
The "PowerConsumptionMax (W)" of LC2 is 1.
    
"LC3 - LittleCore3" is a Core.
The "CoreNumber" of LC3 is 7.
The "MemorySizeMax (KB)" of LC3 is 512.
The "PowerConsumptionMax (W)" of LC3 is 1.
    
"LC4 - LittleCore4" is a Core.
The "CoreNumber" of LC4 is 8.
The "MemorySizeMax (KB)" of LC4 is 512.
The "PowerConsumptionMax (W)" of LC4 is 1.

SHMC is composed of "BC1 - BigCore1", "BC2 - BigCore2", "BC3 - BigCore3", "BC4 - BigCore4"
SHMC is composed of "LC1 - LittleCore1", "LC2 - LittleCore2", "LC3 - LittleCore3", "LC4 - LittleCore4"

sw_uav.xml parsed
BC1 - BigCore1 is a Core
BC2 - BigCore2 is a Core
BC3 - BigCore3 is a Core
BC4 - BigCore4 is a Core
LC1 - LittleCore1 is a Core
LC2 - LittleCore2 is a Core
LC3 - LittleCore3 is a Core
LC4 - LittleCore4 is a Core
SHMC - Heterogeneous multi-core architecture platform is composed of BC1 - BigCore1
SHMC - Heterogeneous multi-core architecture platform is composed of BC2 - BigCore2
SHMC - Heterogeneous multi-core architecture platform is composed of BC3 - BigCore3
SHMC - Heterogeneous multi-core architecture platform is composed of BC4 - BigCore4
SHMC - Heterogeneous multi-core architecture platform is composed of LC1 - LittleCore1
SHMC - Heterogeneous multi-core architecture platform is composed of LC2 - LittleCore2
SHMC - Heterogeneous multi-core architecture platform is composed of LC3 - LittleCore3
SHMC - Heterogeneous multi-core architecture platform is composed of LC4 - LittleCore4
Attribute CoreNumber for BC1 - BigCore1 with value 1
Attribute MemorySizeMax (KB) for 

In [8]:
%%jarvis
with sw_uav
show decomposition UAV

sw_uav.xml parsed
Decomposition Diagram for UAV generated


Overview :


![figure](./diagrams/Diagram9095742045.svg)

In [9]:
%%jarvis
with sw_uav
# Req 1: A task Ti is deployed on one and only one core
A Core shall host only one function

sw_uav.xml parsed
Requirement identified about Core: A Core shall host only one function



Please give a requirement name:  REQ_1


REQ_1 is a requirement
Requirement REQ_1 is satisfied by Type Core
sw_uav.xml updated


In [10]:
%%jarvis
with sw_uav
What is BC1 ?

sw_uav.xml parsed


,0
type,Core
alias,BC1
derived from object,none
parent,SHMC - Heterogeneous multi-core architecture platform
child list,none
requirement list,none
PowerConsumptionMax (W),3
MemorySizeMax (KB),2000
CoreNumber,1
inherited requirement list,REQ_1


In [11]:
%%jarvis
with sw_uav
# Req 2: Some tasks can only be performed by certain cores according to:
# * T1 : 1 ou 5
# * T2 : 2 ou 6
# * T6 : 3 ou 7
The function F2 shall run either on the BC1 or on the LC1.
The function F3 shall run either on the BC2 or on the LC2.
THe function F7 shall run either on the BC3 or on the LC3.

sw_uav.xml parsed
Requirement identified about F2 - To identify picture: The function F2 shall run either on the BC1 or on the LC1
[WARNING] Function F2 - To identify picture is not allocated to Functional element BC1 - BigCore1
[WARNING] Function F2 - To identify picture is not allocated to Functional element LC1 - LittleCore1


Please give a requirement name:  REQ_2_a


Requirement identified about F3 - To appply Filter 1: The function F3 shall run either on the BC2 or on the LC2
[WARNING] Function F3 - To appply Filter 1 is not allocated to Functional element BC2 - BigCore2
[WARNING] Function F3 - To appply Filter 1 is not allocated to Functional element LC2 - LittleCore2


Please give a requirement name:  REQ_2_b


Requirement identified about F7 - To predict trajectory: THe function F7 shall run either on the BC3 or on the LC3
[WARNING] Function F7 - To predict trajectory is not allocated to Functional element BC3 - BigCore3
[WARNING] Function F7 - To predict trajectory is not allocated to Functional element LC3 - LittleCore3


Please give a requirement name:  REQ_2_c


REQ_2_a is a requirement
REQ_2_b is a requirement
REQ_2_c is a requirement
Requirement REQ_2_a is satisfied by Function F2 - To identify picture
Requirement REQ_2_b is satisfied by Function F3 - To appply Filter 1
Requirement REQ_2_c is satisfied by Function F7 - To predict trajectory
sw_uav.xml updated


In [12]:
%%jarvis
with sw_uav
# Req 3: Tasks T1 and T3 as well as Task T2 and T4 must necessarily be deployed on the same core. 
# Similarly, Task T0 and T1 must necessarily be deployed on different cores
The function F2 shall run on the same Core as the function F4.
The function F3 shall run on the same Core as the function F5.
The function F1 shall run on a different Core than the function F2.

sw_uav.xml parsed
Requirement identified about F2 - To identify picture: The function F2 shall run on the same Core as the function F4


Please give a requirement name:  REQ_3_a


Requirement identified about F3 - To appply Filter 1: The function F3 shall run on the same Core as the function F5


Please give a requirement name:  REQ_3_b


Requirement identified about F1 - To capture frame: The function F1 shall run on a different Core than the function F2


Please give a requirement name:  REQ_3_c


REQ_3_a is a requirement
REQ_3_b is a requirement
REQ_3_c is a requirement
Requirement REQ_3_a is satisfied by Function F2 - To identify picture
Requirement REQ_3_a is satisfied by Type Core
Requirement REQ_3_a is satisfied by Function F4 - To appply Filter 2
Requirement REQ_3_b is satisfied by Function F3 - To appply Filter 1
Requirement REQ_3_b is satisfied by Type Core
Requirement REQ_3_b is satisfied by Function F5 - To identify objects
Requirement REQ_3_c is satisfied by Function F1 - To capture frame
Requirement REQ_3_c is satisfied by Type Core
Requirement REQ_3_c is satisfied by Function F2 - To identify picture
sw_uav.xml updated
